Install the Azure Machine Learning SDK with AutoML.

In [ ]:
%pip install --upgrade --force-reinstall -r https://aka.ms/automl_linux_requirements.txt

Install the Azure ML Interpretability library.

In [ ]:
%pip install azureml-interpret

Install support for the explainability dashboard.

In [ ]:
%pip install plotly

In [ ]:
%pip install interpret-community[visualization]

Provide information to connect to the Azure Machine Learning workspace.

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.train.automl.run import AutoMLRun
import azureml.train.automl
import azureml.train.automl.runtime
import numpy as np
import pandas as pd
import matplotlib.pylab as plt 

#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- subscription you are using for this hands-on lab

#Replace the name below with the name of your resource group 
resource_group = "MCW-Machine-Learning"

#Replace the name below with the name of your Azure Machine Learning workspace
workspace_name = "ai-in-a-day"

experiment_name = 'Car-Maintenance'
dataset_name = 'CarMaintenance'

## Connect to the Azure Machine Learning workspace

**Important note for workspace authentication**

If you have access to multiple tenants, you may need to import the `InteractiveLoginAuthentication` class and explicitly define what tenant you are targeting. Uncomment the first two lines of the cell below to force interactive authentication.

For mode details, see https://docs.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication.

In [ ]:
#from azureml.core.authentication import InteractiveLoginAuthentication
#interactive_auth = InteractiveLoginAuthentication(tenant_id="<azure_ad_tenant_id>")

# Connect to the Azure ML Workspace
ws = Workspace(subscription_id, resource_group, workspace_name)

# In case you previously used a different name for the Azure Machine Learning experiment, replace the name below with that name.
experiment = Experiment(ws, experiment_name)

# Connect to the AutoML experiment and list the runs associated with the experiment.
list(experiment.get_runs())

Identify the latest run with a status of `Completed`, and use its `Id` to get the best model.

In [ ]:
# Replace `<automl_run_id>` below with the `Id` of the latest completed run
automl_run = AutoMLRun(experiment, run_id = '<automl_run_id>')

best_run, best_model = automl_run.get_output()
best_model

Load the dataset used to train the model with AutoML.

In [ ]:
car_maintenance_ds = Dataset.get_by_name(ws, dataset_name)
car_maintenance_df = car_maintenance_ds.to_pandas_dataframe()
car_maintenance_df

In [ ]:
X = car_maintenance_df.drop(columns=['has-issue-in-first-10k'])
y_true = car_maintenance_df['has-issue-in-first-10k'].astype(int)
y_pred = best_model.predict(car_maintenance_df.drop(columns=['has-issue-in-first-10k'])).astype(int)

In [ ]:
np.unique(y_pred)

In [ ]:
from fairlearn.widget import FairlearnDashboard
FairlearnDashboard(sensitive_features=car_maintenance_df['gender'], 
                   sensitive_feature_names=['gender'],
                   y_true=y_true,
                   y_pred=y_pred)

In [ ]:
X

In [ ]:
#  Create a dictionary of model(s) you want to assess for fairness 
sf = { 'Gender': X['gender'] }

y_pr = { 'lr_reg_id': y_pred }

from fairlearn.metrics._group_metric_set import _create_group_metric_set

dash_dict = _create_group_metric_set(y_true=y_true,
                                    predictions=y_pr,
                                    sensitive_features=sf,
                                    prediction_type='binary_classification')




In [ ]:
from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

In [ ]:
run = experiment.start_logging()
# run.start()

# Upload the dashboard to Azure Machine Learning
try:
    dashboard_title = "Fairness insights of Logistic Regression Classifier"
    # Set validate_model_ids parameter of upload_dashboard_dictionary to False if you have not registered your model(s)
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title,
                                           validate_model_ids=False)
    print("\nUploaded to id: {0}\n".format(upload_id))

    # To test the dashboard, you can download it back and ensure it contains the right information
    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
finally:
    run.complete()
